In [1]:
%pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color
from skimage.transform import resize
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score, accuracy_score

### Data Loading and Processing (Adapt for your dataset)

In [3]:
class NucleiDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_size=(128, 128)):
        self.image_size = image_size
        self.image_paths = []
        self.mask_paths = []
        self.image_filenames = []
        image_files = sorted(os.listdir(image_dir))
        mask_files = sorted(os.listdir(mask_dir))

        for img_file, mask_file in zip(image_files, mask_files):
            img_path = os.path.join(image_dir, img_file)
            mask_path = os.path.join(mask_dir, mask_file)
            self.image_paths.append(img_path)
            self.mask_paths.append(mask_path)
            self.image_filenames.append(img_file)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
        img_filename = self.image_filenames[idx]

        img = io.imread(img_path)
        mask = io.imread(mask_path)

        if img.ndim == 3 and img.shape[2] == 4:
            img = img[:, :, :3]  # Drop alpha if RGBA

        if mask.ndim == 3:
            if mask.shape[2] == 4:
                mask = mask[:, :, :3]
            mask = color.rgb2gray(mask)
        elif mask.ndim == 2:
            pass  # already grayscale

        img_resized = resize(img, self.image_size, anti_aliasing=True)
        if img_resized.ndim == 3:
            img_gray = color.rgb2gray(img_resized)
        else:
            img_gray = img_resized

        mask_resized = resize(mask, self.image_size, anti_aliasing=True)
        mask_binary = mask_resized > 0.5

        return torch.tensor(np.expand_dims(img_gray.astype(np.float32), axis=0)).to(device), \
               torch.tensor(np.expand_dims(mask_binary.astype(np.float32), axis=0)).to(device), \
               resize(img, self.image_size, anti_aliasing=True), \
               resize(mask, self.image_size, anti_aliasing=True), \
               img_filename

### Hybrid Attention Module (HAM)

In [4]:
class HybridAttention(nn.Module):
    def __init__(self, in_channels):
        super(HybridAttention, self).__init__()
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, in_channels // 8, kernel_size=1).to(device),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels // 8, in_channels, kernel_size=1).to(device),
            nn.Sigmoid()
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 8, kernel_size=1).to(device),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels // 8, 1, kernel_size=3, padding=1).to(device),
            nn.Sigmoid()
        )

    def forward(self, x):
        channel_attn = self.channel_attention(x)
        spatial_attn = self.spatial_attention(x)
        channel_refined = x * channel_attn
        spatial_refined = x * spatial_attn
        # Novel combination: Element-wise multiplication of channel and spatial refined features
        attention_out = channel_refined * spatial_refined
        return x + attention_out

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels_1x1, red_channels_3x3, out_channels_3x3, red_channels_5x5, out_channels_5x5, pool_proj):
        super(InceptionModule, self).__init__()
        self.branch1 = nn.Conv2d(in_channels, out_channels_1x1, kernel_size=1).to(device)
        self.branch2_red = nn.Conv2d(in_channels, red_channels_3x3, kernel_size=1).to(device)
        self.branch2_conv = nn.Conv2d(red_channels_3x3, out_channels_3x3, kernel_size=3, padding=1).to(device)
        self.branch3_red = nn.Conv2d(in_channels, red_channels_5x5, kernel_size=1).to(device)
        self.branch3_conv = nn.Conv2d(red_channels_5x5, out_channels_5x5, kernel_size=5, padding=2).to(device)
        self.branch4_pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.branch4_proj = nn.Conv2d(in_channels, pool_proj, kernel_size=1).to(device)

    def forward(self, x):
        branch1_out = F.relu(self.branch1(x))
        branch2_out = F.relu(self.branch2_conv(F.relu(self.branch2_red(x))))
        branch3_out = F.relu(self.branch3_conv(F.relu(self.branch3_red(x))))
        branch4_out = F.relu(F.relu(self.branch4_proj(self.branch4_pool(x))))
        outputs = [branch1_out, branch2_out, branch3_out, branch4_out]
        return torch.cat(outputs, 1)

class HMSAModule(nn.Module):
    def __init__(self, in_channels, out_channels_1x1, red_channels_3x3, out_channels_3x3, red_channels_5x5, out_channels_5x5, pool_proj):
        super(HMSAModule, self).__init__()
        self.inception = InceptionModule(in_channels, out_channels_1x1, red_channels_3x3, out_channels_3x3, red_channels_5x5, out_channels_5x5, pool_proj).to(device)
        inception_output_channels = out_channels_1x1 + out_channels_3x3 + out_channels_5x5 + pool_proj
        self.attention = HybridAttention(inception_output_channels).to(device)
        if inception_output_channels != in_channels:
            self.projection = nn.Conv2d(inception_output_channels, in_channels, kernel_size=1).to(device)
        else:
            self.projection = None

    def forward(self, x):
        inception_out = self.inception(x)
        attention_out = self.attention(inception_out)
        if self.projection is not None:
            attention_out = self.projection(attention_out)
        hmsam_output = x + attention_out
        return hmsam_output

### HMSAM-UNet Model

In [5]:

class HMSAM_UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(HMSAM_UNet, self).__init__()
        def double_conv(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, kernel_size=3, padding=1).to(device),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_c, out_c, kernel_size=3, padding=1).to(device),
                nn.ReLU(inplace=True)
            )

        self.enc1 = double_conv(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = double_conv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = double_conv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.enc4 = double_conv(256, 512)
        self.pool4 = nn.MaxPool2d(2)

        self.hmsam = HMSAModule(in_channels=512,
                                out_channels_1x1=128,
                                red_channels_3x3=96, out_channels_3x3=192,
                                red_channels_5x5=32, out_channels_5x5=64,
                                pool_proj=128)

        self.upconv4 = nn.ConvTranspose2d(512, 512, kernel_size=2, stride=2).to(device)
        self.dec4 = double_conv(1024, 512)

        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2).to(device)
        self.dec3 = double_conv(512, 256)

        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2).to(device)
        self.dec2 = double_conv(256, 128)

        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2).to(device)
        self.dec1 = double_conv(128, 64)

        self.outc = nn.Conv2d(64, out_channels, kernel_size=1).to(device)

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool1(enc1))
        enc3 = self.enc3(self.pool2(enc2))
        enc4 = self.enc4(self.pool3(enc3))
        bridge = self.pool4(enc4)

        hmsam_out = self.hmsam(bridge)

        dec4 = self.dec4(torch.cat([self.upconv4(hmsam_out), enc4], dim=1))
        dec3 = self.dec3(torch.cat([self.upconv3(dec4), enc3], dim=1))
        dec2 = self.dec2(torch.cat([self.upconv2(dec3), enc2], dim=1))
        dec1 = self.dec1(torch.cat([self.upconv1(dec2), enc1], dim=1))

        out = torch.sigmoid(self.outc(dec1))
        return out

### Training

In [6]:
def train_unet(model, train_loader, num_epochs=10, lr=1e-3):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for img_tensor, mask_tensor, _, _, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            img_tensor, mask_tensor = img_tensor.to(device), mask_tensor.to(device)
            optimizer.zero_grad()
            output = model(img_tensor)
            loss = criterion(output, mask_tensor)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")
    return model

### Prediction

In [7]:
def predict_unet(model, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    predictions = []
    original_images = []
    ground_truth_masks = []
    image_filenames = []
    with torch.no_grad():
        for img_tensor, mask_tensor, original_img, original_mask, filename in tqdm(test_loader, desc="Predicting"):
            img_tensor = img_tensor.to(device)
            output = model(img_tensor).cpu().numpy()
            predictions.append((output[0], filename[0]))
            original_images.extend(original_img)
            ground_truth_masks.extend(original_mask)
            image_filenames.extend(filename)
    return predictions, original_images, ground_truth_masks, image_filenames

### Evaluation

In [8]:
def evaluate_predictions(predictions, gt_masks):
    dice_scores = []
    iou_scores = []
    precisions = []
    recalls = []
    accuracies = []

    for pred_mask, fname in predictions:
        gt_mask = gt_masks.get(fname)
        if gt_mask is None:
            print(f"Ground truth for {fname} not found.")
            continue

        pred_mask_binary = (pred_mask > 0.5).flatten().astype(np.uint8)
        gt_mask_array = gt_mask.cpu().numpy()
        gt_mask_binary = (gt_mask_array > 0.5).flatten().astype(np.uint8)

        dice = f1_score(gt_mask_binary, pred_mask_binary)
        iou = jaccard_score(gt_mask_binary, pred_mask_binary)
        precision = precision_score(gt_mask_binary, pred_mask_binary, zero_division=0)
        recall = recall_score(gt_mask_binary, pred_mask_binary, zero_division=0)
        accuracy = accuracy_score(gt_mask_binary, pred_mask_binary)

        dice_scores.append(dice)
        iou_scores.append(iou)
        precisions.append(precision)
        recalls.append(recall)
        accuracies.append(accuracy)

    print("\nEvaluation Metrics:")
    print(f"Dice Score (F1):  {np.mean(dice_scores):.4f}")
    print(f"IoU Score:        {np.mean(iou_scores):.4f}")
    print(f"Precision:        {np.mean(precisions):.4f}")
    print(f"Recall:           {np.mean(recalls):.4f}")
    print(f"Accuracy:         {np.mean(accuracies):.4f}")

### Visualization

In [9]:
def visualize_results(original_images, ground_truth_masks, predictions, num_samples=5, save_dir='results/hmsam_unet'):
    os.makedirs(save_dir, exist_ok=True)
    for i in range(min(num_samples, len(original_images))):
        plt.figure(figsize=(15, 5))

        # Original image
        plt.subplot(1, 3, 1)
        img_to_plot = original_images[i].squeeze(0)
        if img_to_plot.ndim == 3:
            plt.imshow(img_to_plot)
        else:
            plt.imshow(img_to_plot, cmap='gray')
        plt.title('Original Image')
        plt.axis('off')

        # Ground truth mask
        plt.subplot(1, 3, 2)
        mask_to_plot = ground_truth_masks[i].squeeze(0)
        if mask_to_plot.ndim == 3:
            plt.imshow(mask_to_plot)
        else:
            plt.imshow(mask_to_plot, cmap='gray')
        plt.title('Ground Truth Mask')
        plt.axis('off')

        # Prediction (binarized output)
        plt.subplot(1, 3, 3)
        plt.imshow(predictions[i][0] > 0.5, cmap='gray')
        plt.title('Prediction')
        plt.axis('off')

        plt.savefig(os.path.join(save_dir, f'prediction_result_{i}.png'))
        plt.close()

### Main Execution

In [ ]:
if __name__ == '__main__':
    train_image_dir = 'NucleiSegmentationDataset/all_images'
    train_mask_dir = 'NucleiSegmentationDataset/merged_masks'
    test_image_dir = 'TestDataset/images'
    test_mask_dir = 'TestDataset/masks'
    # train_image_dir = '/content/drive/MyDrive/NucleiSegmentation/TestDataset/images'
    # train_mask_dir = '/content/drive/MyDrive/NucleiSegmentation/TestDataset/masks'

    train_dataset = NucleiDataset(train_image_dir, train_mask_dir)
    test_dataset = NucleiDataset(test_image_dir, test_mask_dir)

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    hmsam_unet_model = HMSAM_UNet(in_channels=1, out_channels=1)

    hmsam_unet_model = train_unet(hmsam_unet_model, train_loader, num_epochs=25, lr=0.001) # Reduced epochs for quicker test

    predictions, original_test_images, original_test_masks, test_filenames = predict_unet(hmsam_unet_model, test_loader)


    gt_masks_for_eval = {}
    for mask, filename in zip(original_test_masks, test_filenames):
        gt_masks_for_eval[filename] = mask

    evaluate_predictions(predictions, gt_masks_for_eval)
    visualize_results(original_test_images, original_test_masks, predictions, num_samples=5)

cpu


Epoch 1/25: 100%|██████████| 167/167 [03:43<00:00,  1.34s/it]


Epoch 1/25, Loss: 0.2846


Epoch 2/25: 100%|██████████| 167/167 [03:42<00:00,  1.33s/it]


Epoch 2/25, Loss: 0.6377


Epoch 3/25: 100%|██████████| 167/167 [03:46<00:00,  1.36s/it]


Epoch 3/25, Loss: 0.1577


Epoch 4/25: 100%|██████████| 167/167 [03:28<00:00,  1.25s/it]


Epoch 4/25, Loss: 0.1155


Epoch 5/25:  81%|████████▏ | 136/167 [03:07<00:48,  1.56s/it]